## Transshipment Problem: Zeron Industries

In [25]:
# import Glop package
from ortools.linear_solver import pywraplp as glp
import lptools as lpt

In [26]:
#Create LP model object
mymodel = glp.Solver('Zeron',glp.Solver.GLOP_LINEAR_PROGRAMMING)

In [27]:
inf = mymodel.infinity()

         # name: supply capacity, objective coefficient list
source = {'A': (75 , [5,8 ]),    # Arnold capacity and shipping cost per unit to each DC
          'S': (75 , [7,4 ])}    # Super Shelf capacity and shipping cost per unit to each DC

transship = {'ZN': [1,5,8 ],    # Zeron-North DC shipping cost per unit to each customer
             'ZS': [3,4,4 ]}    # Zeron-South DC shipping cost per unit to each customer

        # name: demand quantity required
demand = {'Z': 50, 'H': 60, 'R': 40}    # weekly customer demand for Zrox, Hewes, and RockRite

mymodel.Objective().SetMinimization()

In [28]:
# create inbound shipment variables from suppliers to dcs and a dictionary to access them
ship = dict()
for s in source:
    ship[s] = dict()
    (b,coeff_lst) = source[s]
    for (t,c) in zip(transship,coeff_lst):
        ship[s][t] = mymodel.NumVar(0,inf, s + "." + t)
        mymodel.Objective().SetCoefficient(ship[s][t], c)

In [29]:
print(ship)

{'A': {'ZN': A.ZN, 'ZS': A.ZS}, 'S': {'ZN': S.ZN, 'ZS': S.ZS}}


In [30]:
# create outbound shipment variables from DCs to Customers and a dictionary to access them
for t in transship:
    ship[t] = dict()
    coeff_lst = transship[t]
    for (d,c) in zip(demand,coeff_lst):
        ship[t][d] = mymodel.NumVar(0,inf, t + "." + d )
        mymodel.Objective().SetCoefficient(ship[t][d], c)

In [31]:
print(ship)

{'A': {'ZN': A.ZN, 'ZS': A.ZS}, 'S': {'ZN': S.ZN, 'ZS': S.ZS}, 'ZN': {'Z': ZN.Z, 'H': ZN.H, 'R': ZN.R}, 'ZS': {'Z': ZS.Z, 'H': ZS.H, 'R': ZS.R}}


In [32]:
# create source/supply constraints
for s in source:
    (b,coeff_lst) = source[s]
    constr = mymodel.Constraint(-inf,b,s )
    for t in transship:
        constr.SetCoefficient(ship[s][t], 1)

In [33]:
lpt.print_model(mymodel)

Variables:
A.ZN, A.ZS, S.ZN, S.ZS, ZN.Z, ZN.H, ZN.R, ZS.Z, ZS.H, ZS.R 

minimize: 5.0*A.ZN + 8.0*A.ZS + 7.0*S.ZN + 4.0*S.ZS + 1.0*ZN.Z + 5.0*ZN.H + 8.0*ZN.R + 3.0*ZS.Z + 4.0*ZS.H + 4.0*ZS.R 

Subject To:
A: 1.0*A.ZN + 1.0*A.ZS <= 75.0
S: 1.0*S.ZN + 1.0*S.ZS <= 75.0

Bounds:
A.ZN >= 0.0
A.ZS >= 0.0
S.ZN >= 0.0
S.ZS >= 0.0
ZN.Z >= 0.0
ZN.H >= 0.0
ZN.R >= 0.0
ZS.Z >= 0.0
ZS.H >= 0.0
ZS.R >= 0.0


In [34]:
# create demand constraints
for d in demand:
    lb = ub = demand[d]
    constr = mymodel.Constraint(lb,ub,d)
    for t in transship:
        constr.SetCoefficient(ship[t][d], 1)

In [35]:
lpt.print_model(mymodel)

Variables:
A.ZN, A.ZS, S.ZN, S.ZS, ZN.Z, ZN.H, ZN.R, ZS.Z, ZS.H, ZS.R 

minimize: 5.0*A.ZN + 8.0*A.ZS + 7.0*S.ZN + 4.0*S.ZS + 1.0*ZN.Z + 5.0*ZN.H + 8.0*ZN.R + 3.0*ZS.Z + 4.0*ZS.H + 4.0*ZS.R 

Subject To:
A: 1.0*A.ZN + 1.0*A.ZS <= 75.0
S: 1.0*S.ZN + 1.0*S.ZS <= 75.0
Z: 1.0*ZN.Z + 1.0*ZS.Z = 50.0
H: 1.0*ZN.H + 1.0*ZS.H = 60.0
R: 1.0*ZN.R + 1.0*ZS.R = 40.0

Bounds:
A.ZN >= 0.0
A.ZS >= 0.0
S.ZN >= 0.0
S.ZS >= 0.0
ZN.Z >= 0.0
ZN.H >= 0.0
ZN.R >= 0.0
ZS.Z >= 0.0
ZS.H >= 0.0
ZS.R >= 0.0


In [36]:
# create transshipment flow balance constraints
for t in transship:
    constr = mymodel.Constraint(0,0,t )
    for s in source:
        constr.SetCoefficient(ship[s][t], 1)
    for d in demand:
        constr.SetCoefficient(ship[t][d], -1)

In [37]:
lpt.print_model(mymodel)

Variables:
A.ZN, A.ZS, S.ZN, S.ZS, ZN.Z, ZN.H, ZN.R, ZS.Z, ZS.H, ZS.R 

minimize: 5.0*A.ZN + 8.0*A.ZS + 7.0*S.ZN + 4.0*S.ZS + 1.0*ZN.Z + 5.0*ZN.H + 8.0*ZN.R + 3.0*ZS.Z + 4.0*ZS.H + 4.0*ZS.R 

Subject To:
A: 1.0*A.ZN + 1.0*A.ZS <= 75.0
S: 1.0*S.ZN + 1.0*S.ZS <= 75.0
Z: 1.0*ZN.Z + 1.0*ZS.Z = 50.0
H: 1.0*ZN.H + 1.0*ZS.H = 60.0
R: 1.0*ZN.R + 1.0*ZS.R = 40.0
ZN: 1.0*A.ZN + 1.0*S.ZN - 1.0*ZN.Z - 1.0*ZN.H - 1.0*ZN.R = 0.0
ZS: 1.0*A.ZS + 1.0*S.ZS - 1.0*ZS.Z - 1.0*ZS.H - 1.0*ZS.R = 0.0

Bounds:
A.ZN >= 0.0
A.ZS >= 0.0
S.ZN >= 0.0
S.ZS >= 0.0
ZN.Z >= 0.0
ZN.H >= 0.0
ZN.R >= 0.0
ZS.Z >= 0.0
ZS.H >= 0.0
ZS.R >= 0.0


In [38]:
#solve model and display results
status = mymodel.Solve()
print('Solution Status =',status)
print('Optimal Value = %.2f' % mymodel.Objective().Value())
for v in mymodel.variables():
    print('%s = %.2f' % (v.name(),v.solution_value()))

Solution Status = 0
Optimal Value = 1150.00
A.ZN = 75.00
A.ZS = 0.00
S.ZN = 0.00
S.ZS = 75.00
ZN.Z = 50.00
ZN.H = 25.00
ZN.R = 0.00
ZS.Z = 0.00
ZS.H = 35.00
ZS.R = 40.00


In [39]:
# display all variable information
print('Variable    LB   Value    UB   Reduced Cost')
for v in mymodel.variables():
    print('%8s  %5.1f  %5.1f  %5.1f  %5.2f' % (v.name(),v.lb(),v.solution_value(),v.ub(),v.reduced_cost()))

Variable    LB   Value    UB   Reduced Cost
    A.ZN    0.0   75.0    inf   0.00
    A.ZS    0.0    0.0    inf   2.00
    S.ZN    0.0    0.0    inf   4.00
    S.ZS    0.0   75.0    inf   0.00
    ZN.Z    0.0   50.0    inf   0.00
    ZN.H    0.0   25.0    inf   0.00
    ZN.R    0.0    0.0    inf   3.00
    ZS.Z    0.0    0.0    inf   3.00
    ZS.H    0.0   35.0    inf   0.00
    ZS.R    0.0   40.0    inf   0.00


In [40]:
# only print nonzero shipment variables
# print('Variable    LB   Value    UB   Reduced Cost')
# for v in mymodel.variables():
#     if v.solution_value() != 0:
#         print('%8s  %5.1f  %5.1f  %5.1f  %5.2f' % (v.name(),v.lb(),v.solution_value(),v.ub(),v.reduced_cost()))

In [41]:
#display constraint information
print('Constraint    LB    Value  UB     Dual')
for (c,lhs) in zip(mymodel.constraints(),mymodel.ComputeConstraintActivities()):
    print('%10s  %5.1f  %5.1f  %5.1f  %5.2f' % (c.name(),c.lb(),lhs,c.ub(),c.dual_value()))

Constraint    LB    Value  UB     Dual
         A   -inf   75.0   75.0   0.00
         S   -inf   75.0   75.0  -2.00
         Z   50.0   50.0   50.0   6.00
         H   60.0   60.0   60.0  10.00
         R   40.0   40.0   40.0  10.00
        ZN    0.0    0.0    0.0   5.00
        ZS    0.0    0.0    0.0   6.00
